# Discover the Higgs with Deep Neural Networks

The input data was created from 13 TeV ATLAS open data available at http://opendata.atlas.cern/release/2020/documentation/index.html

For more information read:<br>
Review of the 13 TeV ATLAS Open Data release, Techn. Ber., All figures including auxiliary figures are available at https://atlas.web.cern.ch/Atlas/GROUPS/PHYSICS/PUBNOTES/ATL-OREACH-PUB-2020-001: CERN, 2020, url: http://cds.cern.ch/record/2707171

The data is measured by the ATLAS detector, one of the four big detectors at the Large Hadron Collider (LHC) at the CERN research center:

<div>
<img src='figures/ATLAS_detector.png' width='700'/>
</div>
ATLAS Experiment © 2008 CERN

The data analyzed in this jupyter notebook measured at a centre-of-mass energy of $\sqrt{s}=13 \text{ TeV}$ with an integrated luminosity of $10 \text{ fb}^{-1}$ in the year 2016. To search for H$\rightarrow$ZZ$\rightarrow$llll events only events with four reconstructed leptons in the final state are included in the given datasets. The events will have a event signature similar to:

<div>
<img src='figures/ATLAS_four_lepton_event.png' width='700'/>
</div>
https://atlas.web.cern.ch/Atlas/GROUPS/PHYSICS/CONFNOTES/ATLAS-CONF-2011-162/

## Simulation and Event Weights

In quantum physics, no concrete process outcomes are predicted, only their probabilities. In order to make a prediction for the measurement at the ATLAS detector, the frequencies must be simulated. For this purpose, random events are generated on the basis of the probability densities and then their respective measurement in the detector is simulated. Generating the expected number of events results in the following distribution for the lepton with the largest transverse momentum. The distribution itself is again split into the different Higgs processes (ggH125_ZZ4lep, VBFH125_ZZ4lep, WH125_ZZ4lep and ZH125_ZZ4lep) and the background processes (llll, Zee, Zmumu and ttbar_lep).
<div>
<img src='figures/event_weights_few_not_applied.png' width='500'/>
</div>

This graphic already offers a good insight into how the data to be measured could be distributed. Unfortunately, the distribution is not very smooth due to the low number of events. To improve the prediction, more events are simulated than would actually be expected in the data. Especially for processes of high interest, like here the Higgs processes, especially many events are generated. The higher statistic results into much smoother predictions.
<div>
<img src='figures/event_weights_all_not_applied.png' width='500'/>
</div>

However, now both the ratios between the different processes have shifted and the prediction of the total events has increased extremely. This distribution of the "raw" simulation events thus no longer corresponds to what can be expected for the actual measurement. To correct this, event weights are applied. Thus, each simulated event will enter the distribution only as fraction of an event given by the respective event weight. This weight depends on the respective simulated process as well as on the kinematic region of the event. In addition, there are also negative weights to compensate for excess simulated events. The result is comparable to the initial distribution but offers a much smoother prediction.
<div>
<img src='figures/event_weights_all_applied.png' width='500'/>
</div>

## Data Preparation

### Load the Data

In [ ]:
# Necessary imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools
from numpy.random import seed
import os

# Random state
random_state = 10
_ = np.random.RandomState(random_state)

In [ ]:
import common

The goal of this lab course is to train a deep neural network to separate Higgs boson signal from background events. The most important signal sample ggH125_ZZ4lep corresponds to the process gg$\rightarrow$H$\rightarrow$ZZ. The dominant background sample is llll resulting from Z and ZZ decays.
After training the DNN model will be used to classify the events of the data samples.

Higgs signal samples:
- ggH125_ZZ4lep
- VBFH125_ZZ4lep
- WH125_ZZ4lep
- ZH125_ZZ4lep

Background samples:
- llll
- Zee
- Zmumu
- ttbar_lep

Data samples:
- data_A
- data_B
- data_C
- data_D

In [ ]:
# Define the input samples
sample_list_signal = ['ggH125_ZZ4lep']
sample_list_background = ['llll']
sample_list_signal = ['ggH125_ZZ4lep', 'VBFH125_ZZ4lep', 'WH125_ZZ4lep', 'ZH125_ZZ4lep']
sample_list_background = ['llll', 'Zee', 'Zmumu', 'ttbar_lep']
# sample_list_measured = ['data_A', 'data_B', 'data_C', 'data_D']

In [ ]:
sample_path = 'input'
# Read all the samples
no_selection_data_frames = {}
for sample in sample_list_signal + sample_list_background:
    no_selection_data_frames[sample] = pd.read_csv(os.path.join(sample_path, sample + '.csv'))

### Input Variables

The input provides several variables to classify the events. Since each event has multiple leptons, they were ordered in descending order based on their transverse momentum. Thus, lepton 1 has the highest transverse momentum, lepton 2 the second highest, and so on. <br>
Most of the given variables can be called low-level, because they represent event or object properties, which can be derived directly from the reconstruction in the detector. In contrast to this are high-level variables, which result from the combination of several low-level variables. In the given dataset the only high-level variables are invariant masses of multiple particles:<br>
$m_{inv} = \sqrt{\left(\sum\limits_{i=1}^{n} E_i\right)^2 - \left(\sum\limits_{i=1}^{n} \vec{p}_i\right)^2}$


List of all available variables:<br>
- Scale and event weight
     - The scaling for a dataset is given by the sum of event weights, the cross section, luminosity and a efficiency scale factor
     - Each event has an additional specific event weight
     - To combine simulated events and finally compare them to data each event has to be scaled by the event weight
     - The weight are not used for training
     - Variable name: `totalWeight`
- Number of jets
     - Jets are particle showers which result primarily from quarks and gluons
     - Variable name: `jet_n`
- Invariant four lepton mass
     - The invariant mass $m_{inv}(l_1, l_2, l_3, l_4)$ is the reconstructed invariant mass of the full four lepton event.<br>
     This variable is to be displayed later but not used for training.
     - Variable name: `lep_m_llll`
- Invariant two lepton mass
     - Invariant masses $m_{inv}(l_i, l_j)$ of all combinations of two leptons
     - Variable names: `lep_m_ll_12`, `lep_m_ll_13`, `lep_m_ll_14`, `lep_m_ll_23`, `lep_m_ll_24`, `lep_m_ll_34`
- Transverse momentum $p_T$ of the leptons
     - The momentum in the plane transverse to the beam axis
     - Variable names: `lep1_pt`, `lep2_pt`, `lep3_pt`, `lep4_pt`
- Lepton azimuthal angle
     - The azimuthal angle $\phi$ is measured in the plane transverse to the beam axis
     - Variable name: `lep1_phi`, `lep2_phi`, `lep3_phi`, `lep4_phi`
- Lepton pseudo rapidity
     - The angle $\theta$ is measured between the lepton track and the beam axis.<br>
     Since this angle is not invariant against boosts along the beam axis, the pseudo rapidity $\eta = - \ln{\tan{\frac{\theta}{2}}}$ is primarily used in the ATLAS analyses
     - Variable names: `lep1_eta`, `lep2_eta`, `lep3_eta`, `lep4_eta`
- Lepton energy
     - The energy of the leptons reconstructed from the calorimeter entries
     - Variable name: `lep1_e`, `lep2_e`, `lep3_e`, `lep4_e`
- Lepton PDG-ID
     - The lepton type is classified by a n umber given by the Particle-Data-Group.<br>
     The lepton types are PDG-ID$(e)=11$, PDG-ID$(\mu)=13$ and PDG-ID$(\tau)=15$
     - Variable name: `lep1_pdgId`, `lep2_pdgId`, `lep3_pdgId`, `lep4_pdgId`
- Lepton charge
     - The charge of the given lepton reconstructed by the lepton track
     - Variable name: `lep1_charge`, `lep2_charge`, `lep3_charge`, `lep4_charge`

### Event Pre-Selection

Before we start with the pre-selection of the input data check the number of events per process.

In [ ]:
# Loop over all processes
for sample in sample_list_signal + sample_list_background:
    # Sum over the weights is equal to the number of expected events
    n_events = sum(no_selection_data_frames[sample]['totalWeight'])
    # Number of raw simulation events
    n_events_raw = len(no_selection_data_frames[sample]['totalWeight'])
    print(f'{sample}: {round(n_events, 2)}; {n_events_raw} (raw)')

Although the final selection of the data is to be performed on the basis of a DNN, a rough pre-selection of the data is still useful.
For this purpose, selection criteria are defined, which return either true or false based on the event kinematics and thus decide whether the respective event is kept or discarded.
Suitable criteria for this analysis are very basic selections that must be clearly fulfilled by H->ZZ->llll processes.

<font color='blue'>
Task:

Find baseline selection criteria that reduce the background while keeping almost all Higgs events. 
</font>


Hint: What lepton types and charges are expected in the final state?

In [ ]:
def cut_dummy(lep1_e):
    """This is an dummy cut which all events pass with a leading lepton energy higher than 50 GeV"""
    return lep1_e > 50

In [ ]:
# Create a copy of the original data frame to investigate later
data_frames = no_selection_data_frames.copy()

# Apply the chosen selection criteria
for sample in sample_list_signal + sample_list_background + sample_list_measured:
    # Dummy selection
    dummy_selection = np.vectorize(cut_dummy)(data_frames[sample].lep1_e)
    data_frames[sample] = data_frames[sample][dummy_selection]

<font color='blue'>
Task:

Check wether your selection criteria have the required effects 
</font>

In [ ]:
# Loop over all processes
for sample in sample_list_signal + sample_list_background:
    # Sum over the weights is equal to the number of expected events
    n_events = 
    # Number of raw simulation events
    n_events_raw = 
    print(f'{sample}: {round(n_events, 2)}; {n_events_raw} (raw)')

If you are happy with you baseline selection continue with the investigation of the data.

### Data Investigation

Before one can decide which variables are suitable for training, one must first get a feel for the input variables.
For this purpose, the input samples are merged into a set of signal events and a set of background events. Afterwards, the behavior of signal and background can be studied in multiple variables.

In [ ]:
# Merge the signal and background data frames
def merge_data_frames(sample_list, data_frames_dic):
    for sample in sample_list:
        if sample == sample_list[0]:
            output_data_frame = data_frames_dic[sample]
        else:
            output_data_frame = pd.concat([output_data_frame, data_frames_dic[sample]], axis=0)
    return output_data_frame

data_frame_signal = merge_data_frames(sample_list_signal, data_frames)
data_frame_background = merge_data_frames(sample_list_background, data_frames)

The function common.plot_hist(variable, data_frame_1, data_frame_2) plots the given variable of the two datasets.
The variable must be a dictionary containing atleast the variable to plot. Additionally one can also specify the binning (list or numpy array) and the xlabel. The created histogram is automatically saved in the plots directory<br>

<font color='blue'>
Task:

Which variable is the most discriminant? Which variables seem not discriminant at all? 
</font>

An example for the transverse momnetum of the leading lepton is given below:

In [ ]:
# leading lepton pt
var_lep1_pt = {'variable': 'lep1_pt',
               'binning': np.linspace(0, 300, 50),
               'xlabel': '$p_T$ (lep 1) [GeV]'}

common.plot_hist(var_lep1_pt, data_frames)
common.plot_normed_signal_vs_background(var_lep1_pt, data_frame_signal, data_frame_background)
plt.show()

<font color='blue'>
Task:

What is the purity in signal events for the given data?
</font>

As one could already see, the number of simulated raw events is significantly higher than the weighted number of expected events. The contribution of a simulated event to the final prediction is thus given by the respective event weight.

<font color='blue'>
Task:

How many raw events are included in each process simulation and what the corresponding total prediction? What the is minimal, median, maximal event weight of each process?
</font><br>

In [ ]:
for sample in sample_list_signal + sample_list_background:
    print(f'{sample}:')
    n_events = 
    n_events_raw = 
    min_weight = 
    med_weight = 
    max_weight = 
    print(f'  Raw events:      {n_events_raw}')
    print(f'  Prediction:      {round(n_events, 2)}')
    print(f'  Minimal weight:  {min_weight}')
    print(f'  Median weight:   {med_weight}')
    print(f'  Maximal weight:  {max_weight}')
    print()

The different processes were simulated with different accuracy. To model 267 $llll$ events more than half a million raw events are used but 96 Z$\rightarrow ll$ is modelled by only 500 events.

Furthermore, it seems that the event weights even go into the negative range. Negative weighted events are produced to compensate overshooting predictions in certain kinematic areas.

## Choice of the Training Setup

In this part of the notebook we will create and train some simple neural networks. The goal is to find the right setup for training, as this is a basic requirement for successful training.

### How to Create and Train a Neural Network

Lets start with a very simple training on the transverse momentum of the leptons. To speed up the training not all processes are considered for the training.

In [ ]:
# The training input variables
training_variables = ['lep1_pt', 'lep2_pt', 'lep3_pt', 'lep4_pt']
# Use only a subset of the full data
sample_subset_signal = ['ggH125_ZZ4lep']
sample_subset_background = ['llll', 'Zee', 'Zmumu']

Extract the training variables, the event weights, and the classification of the events. The classification is 0 for background processes and 1 for Higgs events. 
To further speed up the setup test only use 40% of the available data.

In [ ]:
values, weights, classification = common.get_dnn_input(data_frames, training_variables, sample_subset_signal, sample_subset_background, 0.4)

In [ ]:
# Show values of 5 random events
random_idx = [1841, 11852, 63217, 78357, 131697]
# Training variables
print('Training values:')
print(values[random_idx])
# Event weights
print('Event weights:')
print(weights[random_idx])
# Classification
print('Classification:')
print(classification[random_idx])

Tensorflow is used to create and train neural networks. For this purpose, the required module is imported and the existing training values and classifications are transformed into a Tensorflow dataset.

In [ ]:
# Import the tensorflow module to create a DNN
import tensorflow as tf
from tensorflow.data import Dataset

In [ ]:
# Convert the data to tensorflow datasets
train_data = Dataset.from_tensor_slices((values, classification))
train_data = train_data.shuffle(len(values), seed=random_state)
# Set the batch size
train_data = train_data.batch(128)

So lets create a neural network consisting of several layers.

A full list of layer types can be found here https://www.tensorflow.org/api_docs/python/tf/keras/layers <br>
Some important examples for layers are:
- Normalization: Shift and scale the training input to have the center at 0 and a standard deviation of 1 
- Dense: a densely connected NN layer
- Dropout: randomly sets input to 0. Can decrease overtraining

The neurons of each layer are activated by the so called activation function. <br>
A full list of provided activation functions can be found here https://www.tensorflow.org/api_docs/python/tf/keras/activations <br>
Some examples are:
- Linear: $linear(x) = x$
- Relu: $relu(x) = max(0, x)$
- Exponential: $exponential(x) = e^x$
- Sigmoid: $sigmoid(x) = 1 / (1 + e^{-x})$

The activation of i-th node in the k-th layer $a_i^{(k)}$ is given by the sum of the activations of the previous layer $a_j^{(k-1)}$ multiplied by the trainings weights $w_{j; i}^{(k)}$ plus a bias $b_i^{(k)}$. For this sum the activation function $f_{activ}$ is called resulting in the activation of the current node.<br>
$a_i^{(k)} = f_{activ}(\sum_j w_{j; i}^{(k)} \cdot a_j^{k-1} + b_i^{(k)})$

Let's create following model:
<div>
<img src='figures/simple_model.png' width='900'/>
</div>

- The input variables are scaled and shifted to have the mean of 0 and the standard deviation of 1. This normalization improves the convergence of the training process.
- Use two dense layers with 60 neurons each. The neurons are activated with the relu function.
- To classify background and signal the last layer should only consist of one neuron which has an activation between 0 and 1. The activation between 0 and 1 if given by the sigmoid function

<font color='blue'>
Task:

How many parameters do you expect for this model?
</font>

Create a normalization layer and adapt it to the training values:

In [ ]:
# Normalization layer
normalization_layer = tf.keras.layers.Normalization(name='Input_normalization')
normalization_layer.adapt(values)

Create a list of model layers and built the model:

In [ ]:
# Create model
model_layers = [
    normalization_layer,
    tf.keras.layers.Dense(60, activation='relu', name='Hidden_layer_1'),
    tf.keras.layers.Dense(60, activation='relu', name='Hidden_layer_2'),
    tf.keras.layers.Dense(1, activation='sigmoid', name='Output_layer'),
]
model = tf.keras.models.Sequential(model_layers, name='simple_model')

Before we start the training of the model lets check the shape and the number of trainable parameters.

In [ ]:
# Display the model's architecture
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True, show_layer_activations=True)

For the training we have to choose an optimization criteria, the loss function. For supervised learning, the loss function returns a value for the divergence between model prediction and the true value. The loss value is high for bad agreement and low for good agreement. Thus, training the model means minimizing the loss function.
The seperartion of signal and background is binary classification problem and thus the binary cross-entropy is used to calculate the loss.
The binary cross-entropy is given by:<br>
$H = -\frac{1}{N} \sum_i^N (y_i^{true} log(y_i^{predict}) + (1 - y_i^{true}) log(1 - y_i^{predict}))$

As you can see the formula distinguishes signal events with $y_i^{true} = 1$ and background events with $y_i^{true} = 0$. In the following figure you can see the cross entropy for a signal and a background event for different prediction values. The closer the prediction value is to the true classification the smaller the crossentropy is. This is exactly the behavior we need for our loss function, and thus training the neural network means minimizing the mean cross-entropy of all training events.
<div>
<img src='figures/binary_cross_entropy.png' width='700'/>
</div>

If you are still wondering why we use the logarithms and why it is called entropy you are encouraged to check out some information theory. The origin of this cross-entropy can be found in the entropy definition in information theory. The entropy of the true distribution is always smaller than the cross-entropy of its estimator. Therefore, minimizing the cross-entropy is equivalent to becomming closer to the true distribution.

In [ ]:
# Loss function
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)

The loss function itself seems not complicated but keep in mind in our example it depends on several thousand parameters of the neural network. Thus, finding the correct parameter combination to minimize the loss is a quite complex problem. The Adam optimizer we will use for the training is a stochastic gradient descent method.

When we converted the training a batch size was chosen `train_data = train_data.batch(128)` which are subsets of the given data. While training the optimizer computes the gradient descent und updates the parameters for each batch.
After updating the parameters for all batches one after the other, this process can be started again from the beginning. A complete run over all training data is called an epoch. After several epochs, the parameters have been adjusted several times for all training data and hopefully a good combination of neural network parameters has been found.

In [ ]:
# Optimizer
adam_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0002, beta_1=0.9)

Configure the model for the training

In [ ]:
# Compilation
model.compile(optimizer=adam_optimizer, loss=loss_fn, metrics=['binary_accuracy'])

Lets train the model for 5 epochs and store the training history in a variable.

In [ ]:
history = model.fit(train_data, epochs=5)

Lets visualize the training progress of the model.

In [ ]:
# Plot the training history
fig, ax = plt.subplots(figsize=(7, 6))
ax.plot(history.history['loss'])
ax.set_xlabel('epoch')
ax.set_ylabel('loss')
_ = plt.show()

<font color='blue'>
Task:

Do you think the training was done after 5 epochs? 
</font>

The model is trained, so the time has come to use it. Let's compare the true classification of some random events and the prediction the model gives for these events.

In [ ]:
# Choose some random events
random_idx = [1841, 11852, 63217, 78357, 131697]
print('Classification:')
print(classification[random_idx])
print('Prediction')
print(model.predict(values[random_idx]))

<font color='blue'>
Task:

Get the prediction for all events and use <code>common.plot_dnn_output(...)</code> to plot the result.
</font>

In [ ]:
# Apply the model for all values
prediction = 

In [ ]:
# Plot the model output
common.plot_dnn_output(prediction, classification)
_ = plt.show()

Use `common.apply_dnn_model(...)` to apply the model for all samples in `data_frames` and add the classification to the data frame. Afterwards the prediction can be plotted just as the other kinematic variables.

In [ ]:
# Apply the model
data_frames_apply_dnn = common.apply_dnn_model(model, data_frames, training_variables, sample_subset_signal + sample_subset_background)

In [ ]:
model_prediction = {'variable': 'model_prediction',
                    'binning': np.linspace(0, 1, 50),
                    'xlabel': 'prediction'}
common.plot_hist(model_prediction, data_frames_apply_dnn, show_data=False)
plt.show()

### Validation Data and Early Stopping

How long do we have to train? The right amount of training is very important for the final performance of the network. If the training was too short the model parameters are poorly adapted to the underlying concepts and the model performance is bad. This is called undertraining. If the training was too long the model will start to learn the training data by heart. This overtraining will lead to a very godd performance at the training data but bad performance on unseen data.

Thus, to test the performance of the model the model has to be applied on unseen data. After each epoch the model is the model is applied on validation data not used for training. If the classification of the validation data has improved for the current epoch the model performance is still improving. If the performance on the validation data does not improve anymore the training can be stopped.

<div>
<img src='figures/over_and_under_training.png' width='700'/>
</div>

In [ ]:
# Import function to split data into train and test data
from sklearn.model_selection import train_test_split

Use again 40% of the available data for training and additionally 20% to validate the model performance.

In [ ]:
# Extract the values, weights and classification of 60% of the data
values, weights, classification = common.get_dnn_input(data_frames, training_variables, sample_list_signal, sample_list_background, 0.6)

In [ ]:
# Split into train and validation data
train_values, val_values, train_classification, val_classification = train_test_split(values, classification, test_size=1/3, random_state=random_state)
train_weights, val_weights = train_test_split(weights, classification, test_size=1/3, random_state=random_state)[:2]

<font color='blue'>
Task:

Let's follow the same strategy as before:
    
- Create tensorflow datasets for training and validation data with 128 events per batch
- Recreate and adapt the normalization layer
- Recreate the tensorflow model with the same number of layers and nodes per layer
- Compile the model
</font>

In [ ]:
# Convert the data to tensorflow datasets


In [ ]:
# Normalization layer

# Create a simple NN
model_layers = [
]
model = tf.keras.models.Sequential(model_layers)
# Compile model


So how to stop the training if the perfomance does not improve anymore?

The answer is early stopping. With early stopping you set a value which should be monitored, in our case the loss on the validation data `val_loss`. Since there can be fluctuations in the tested model performance, it is recommended to use a certain patience after which the training should be stopped. If we set `patience=5` the training is stopped if the `va_loss` has not improved for 5 epochs. Since the model performance has potentionally decreased during this 5 epochs set `restore_best_weights=True` to restore the model state with the best performance.

In [ ]:
# Early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
# Train model with early stopping for the validation data performance
history = model.fit(train_data, validation_data=val_data, callbacks=[early_stopping], epochs=1000)

<font color='blue'>
Task:

Plot the training loss (<code>history.history['loss']</code>) and validation loss (<code>history.history['val_loss']</code>) of the training history. Describe what behavior you can observe for each performance trend.
</font>

In [ ]:
# Plot the training history
fig, ax = plt.subplots(figsize=(7, 6))

ax.set_xlabel('epoch')
ax.set_ylabel('loss')
ax.legend()
_ = plt.show()

The model itself has already an implemented evaluation function. When a tensorflow set is provided it returns the loss and accuracy on this dataset.

In [ ]:
# Evaluate model on training
model_train_evaluation = model.evaluate(train_data)

print(f'train loss = {round(model_train_evaluation[0], 5)}\ttrain accuracy = {round(model_train_evaluation[1], 5)}')

<font color='blue'>
Task:

Evaluate the model on validation data and compare the results to the validation on training data.
</font>

In [ ]:
# Evaluate model on validation data


Now lets apply the model on the train and validation data and plot the classification.

In [ ]:
# Apply the model for training and validation values
train_prediction = model.predict(train_values)
val_prediction = model.predict(val_values)

In [ ]:
# Plot the model output
common.plot_dnn_output(train_prediction, train_classification, val_prediction, val_classification)
_ = plt.show()

As you can see the classification by the model on traning and validation data is very consistent. This is great :)<br>
If we would see a significant difference in train and validation classification this would be a clear sign for overtraining.

<font color='blue'>
Task:

Use <code>common.apply_dnn_model(...)</code> to apply the model for all samples in <code>data_frames</code> and plot the classification.
</font>

In [ ]:
# Apply the model
data_frames_apply_dnn = 

In [ ]:
model_prediction = {'variable': 'model_prediction',
                    'binning': np.linspace(0, 1, 50),
                    'xlabel': 'prediction'}
common.plot_hist(model_prediction, data_frames_apply_dnn, show_data=False)
plt.show()

<font color='blue'>
Task:

The llll events are mostly classified as background and the Higgs events tend to the signal classification. However, the classification of Zee and Zmumu events seems to be quite random. What could be the reason for this?
</font>

### Event Weights

The simulated events are multiplied by weight factors for the final prediction of measured data. Thus, events with large weights are more important for the prediction than events with small weights. In order to take this into account for the training, the event weights `w_i` can be included in the calculation of the loss:<br>
$H = -\frac{1}{N} \sum_i^N w_i (y_i^{true} log(y_i^{predict}) + (1 - y_i^{true}) log(1 - y_i^{predict}))$

<font color='blue'>
Task:

And another time follow the same strategy as before:
    
- Create tensorflow datasets for training and validation data with 128 events per batch. But this time also include the event weights
- Recreate and adapt the normalization layer
- Recreate the tensorflow model with the same number of layers and nodes per layer
- Compile the model
- Train the model
- Plot the classification and do the evaluation
</font>

In [ ]:
# Convert the data to tensorflow datasets
train_data = Dataset.from_tensor_slices((train_values, train_classification, train_weights))
train_data = train_data.shuffle(len(train_data), seed=random_state)
train_data = train_data.batch(128)
val_data = Dataset.from_tensor_slices((val_values, val_classification, val_weights))
val_data = val_data.shuffle(len(val_data), seed=random_state)
val_data = val_data.batch(128)

In [ ]:
# Normalization layer

# Create a simple NN
model_layers = [
]
model = tf.keras.models.Sequential(model_layers)
# Compile model now with the weighted metric
model.compile(optimizer=adam_optimizer, loss=loss_fn, weighted_metrics=['binary_accuracy'])

In [ ]:
# Train model
history = model.fit()

In [ ]:
# Plot the training history


In [ ]:
# Apply the model for training and validation values

# Plot the model output


In [ ]:
# Evaluate the model on training and validation data


In [ ]:
# Apply the model
data_frames_apply_dnn = common.apply_dnn_model(model, data_frames, training_variables, sample_subset_signal + sample_subset_background)
model_prediction = {'variable': 'model_prediction',
                    'binning': np.linspace(0, 1, 50),
                    'xlabel': 'prediction'}
common.plot_hist(model_prediction, data_frames_apply_dnn, show_data=False)
plt.show()

<font color='blue'>
Task:

What differences can you see on the loss, accuarcy and classification distribution? Explain these differences.
</font>

In [ ]:
# Check the event weights
mean_signal_weight = weights[classification > 0.5].mean()
mean_background_weight = weights[classification < 0.5].mean()
print(f'The mean weight for signal events is {mean_signal_weight} and for background events {mean_background_weight}')

#### Reweight the weights of the simulated data

Introducing the event weights caused the problem that the overall background dominates the training. With the event weights of the simulated data the loss is already quite low if all data is consequently classified close to background.<br>
$H = -\frac{1}{N} \sum_i^N w_i (y_i^{true} log(y_i^{predict}) + (1 - y_i^{true}) log(1 - y_i^{predict}))$

To solve this problem we should reweight the weights to have the same effect on training for signal and background events.

<font color='blue'>
Task:

Write a function that reweights the weights:<br>
1. Take the absolute value of the weight to not run into problems with negative weights
2. Split the weights into signal and background weights (by setting the other weights in the array to zero)
3. Scale the signal weights to have the total sum as the background weights
4. Merge the background and scaled signal weights
5. Scale all weights to have a mean weight of 1
</font><br>

In [ ]:
def reweight_weights(weights, classification):
    # Take the absolute value of the weight
    weights_abs = 
    # Split in signal and background weights
    weights_signal = weights_abs*classification
    weights_background = weights_abs*(1 - classification)
    # Scale the signal events
    weights_signal_scaled = 
    # Merge the signal and background events
    weights_reweighted = weights_background + weights_signal_scaled
    # Normalize mean weight to one
    weights_reweighted =
    return weights_reweighted

<font color='blue'>
Task:

Test your reweighting function. The mean of all weights should be 1 and the sum of signal weights and background weights should be equal.
</font>

In [ ]:
weights_reweighted = reweight_weights(weights, classification)
signal_weights_reweighted = weights_reweighted[classification > 0.5]
background_weights_reweighted = weights_reweighted[classification < 0.5]

print(f'Mean weight: {}')
print(f'Signal weight sum: {}')
print(f'Background weight sum: {}')

<font color='blue'>
Task:

Run the training and plot the classification and evaluate the model.
</font>

In [ ]:
# Get reweighted weights
train_weights_reweighted = reweight_weights(train_weights, train_classification)
val_weights_reweighted = reweight_weights(val_weights, val_classification)

# Convert the data to tensorflow datasets
train_data = Dataset.from_tensor_slices((train_values, train_classification, train_weights_reweighted))
train_data = train_data.shuffle(len(train_data), seed=random_state)
train_data = train_data.batch(128)
val_data = Dataset.from_tensor_slices((val_values, val_classification, val_weights_reweighted))
val_data = val_data.shuffle(len(val_data), seed=random_state)
val_data = val_data.batch(128)

In [ ]:
# Normalization layer
normalization_layer = tf.keras.layers.Normalization()
normalization_layer.adapt(train_values)
# Create a simple NN
model_layers = [
    normalization_layer,
    tf.keras.layers.Dense(60, activation='relu'),
    tf.keras.layers.Dense(60, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
]
model = tf.keras.models.Sequential(model_layers)
# Compile model
model.compile(optimizer=adam_optimizer, loss=loss_fn, weighted_metrics=['binary_accuracy'])
# Train model
history = model.fit(train_data, validation_data=val_data, callbacks=[early_stopping], epochs=1000)

In [ ]:
# Plot the training history
fig, ax = plt.subplots(figsize=(7, 6))
ax.plot(history.history['loss'], label='training')
ax.plot(history.history['val_loss'], label='validation')
ax.set_xlabel('epoch')
ax.set_ylabel('loss')
ax.legend()
_ = plt.show()

In [ ]:
# Apply the model for training and validation values

# Plot the model output


In [ ]:
# Apply the model
data_frames_apply_dnn = common.apply_dnn_model(model, data_frames, training_variables, sample_subset_signal + sample_subset_background)
model_prediction = {'variable': 'model_prediction',
                    'binning': np.linspace(0, 1, 50),
                    'xlabel': 'prediction'}
common.plot_hist(model_prediction, data_frames_apply_dnn, show_data=False)
plt.show()

<font color='blue'>
Task:

Compare this classification plot with the one resulting from a training without event weights.
</font>

## Cross-Validation

If you have a look on the training history you see fluctuations in the validation loss. Furthermore, the validation dataset also has a limited size, making it potentially not completely representative for validation.

So how should one evaluate the performance of a model and compare two models?

A commonly used method to evaluate the model performance is k-fold.
The training data is split several times with non-overlapping validation sets. On each split a model is trained and validated on the corresponding validation data. This results into several independently trained models with same size and setup validated on different datasets. Thus, one is able to calculate the mean performance of the resulting models.
<div>
<img src='figures/kFold.png' width='500'/>
</div>

In [ ]:
# import the kFold module
from sklearn.model_selection import KFold

Use kFold to split the data 3 times in 2/3 training and 1/3 validation data each randomly shuffled.

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=3, shuffle=True, random_state=random_state)

Now use kFold to train several models in a for loop. In each loop you have to create a new model and its setup.

<font color='blue'>
Task:

Fill the missing parts in the for loop:
- Reweight the weights
- Convert the values and classification into tensorflow datasets
- Create a model with normalization layer and 2 hidden layers with 60 nodes each
- Compile the model
- Train the model with early stopping
- Evaluate the model on the training and evaluation data
</font><br>

In [ ]:
# Store the models and their training history
kfold_history = []
kfold_model = []
# Store the evaluation on training and validation data
kfold_train_eval_loss = []
kfold_train_eval_acc = []
kfold_val_eval_loss = []
kfold_val_eval_acc = []
split_idx = 1
for train_indices, val_indices in kfold.split(values):
    print(f'Use fold {split_idx}')
    split_idx += 1
    # Get train and validation data 
    train_values = values[train_indices]
    train_classification = classification[train_indices]
    train_weights = weights[train_indices]
    val_values = values[val_indices]
    val_classification = classification[val_indices]
    val_weights = weights[val_indices]
    # Get reweighted weights
    train_weights_reweighted = 
    val_weights_reweighted = 
    # Get train and validation datasets 


    # Normalization layer
    
    # Create a simple NN
    model_layers = [
    ]
    model = tf.keras.models.Sequential(model_layers)
    # Compile model
    

    # Train model
    history = 

    # Append to list
    kfold_history.append(history)
    kfold_model.append(model)

    # Evaluate model on training and validation data
    model_train_evaluation = 
    model_val_evaluation = 
    kfold_train_eval_loss.append(model_train_evaluation[0])
    kfold_train_eval_acc.append(model_train_evaluation[1])
    kfold_val_eval_loss.append(model_val_evaluation[0])
    kfold_val_eval_acc.append(model_val_evaluation[1])

Lets plot the training history of the three models.

In [ ]:
# Plot the training history
fig, ax = plt.subplots(figsize=(7, 6))
color_list = ['r', 'g', 'b']
for k_fold_idx, (history, color) in enumerate(zip(kfold_history, color_list)):
  ax.plot(history.history['loss'], color=color, label=f'{k_fold_idx} training')
  ax.plot(history.history['val_loss'], color=color, ls='--', label=f'{k_fold_idx} val')
ax.set_xlabel('epoch')
ax.set_ylabel('loss')
ax.legend()
_ = plt.show()

<font color='blue'>
Task:

Calculate the mean and std of the validation loss.
</font>

## Save and Load a Model

As you have seen training a model takes some time. So retrain a model each time you need it is definitely not the way to go. Instead one should save the model after the training and load it for application.

Saving and loading a model is very straight forward with:<br>
Use `model.save('path/to/location', save_format='h5')` to save a model <br>
and `model = tf.keras.models.load_model('path/to/location')` to load the model

Lets save the three models of the cross validation

In [ ]:
# Loop over all models
for idx, model in enumerate(kfold_model):
    # Save the model
    model.save(f'models/test_crossval_model_{idx}')

Now load one of the models and plot its summary

In [ ]:
model = tf.keras.models.load_model('models/test_crossval_model_0')

model.summary()

## Train your own Model

Now it is your turn to create your own model to hunt for the Higgs boson. For the beginning, we will have a look on a baseline model. This baseline model does already a good job in the classification and it is up to you to create a better one.

<font color='blue'>
Task:

1. Use reweighted event weights for your training
2. Choose a setup for your model
3. Train your model with early stopping
4. Plot the training history and binary classification on training and validation data
5. check your validation loss <br>
    If the validation loss is not better than for the baseline model by two standard deviations of the baseline validation loss: go back to step 2.
6. Validate your results with cross validation and calculate the mean validation loss and its standard deviation
    If the loss is not significantly better than for the baseline model: go back to step 2.
7. Save your models and training plots
8. Plot the training history

Document what you can observe for your own model.
</font>

### Baseline Model

Lets load the baseline models. The baseline model was trained with cross validation on 40% of the data for training and 20% of the total data for validation.

The val loss of the model is $0.27785 \pm 0.01361$.

In [ ]:
baseline_models = []
for idx in range(3):
    print(f'baseline_models/crossval_model_{idx}')
    model = tf.keras.models.load_model(f'baseline_models/crossval_model_{idx}')
    baseline_models.append(model)

### Training input

<font color='blue'>
Task:

Train your model for all processes on all <u><b>low level</b></u> variables.
</font>

In [ ]:
# Use all processes
sample_list_signal = ['ggH125_ZZ4lep', 'VBFH125_ZZ4lep', 'WH125_ZZ4lep', 'ZH125_ZZ4lep']
sample_list_background = ['llll', 'Zee', 'Zmumu', 'ttbar_lep']

In [ ]:
# The training input variables
training_variables = ['lep1_pt', 'lep2_pt', 'lep3_pt', 'lep4_pt']
training_variables += ['lep1_e', 'lep2_e', 'lep3_e', 'lep4_e']
training_variables += ['lep1_charge', 'lep2_charge', 'lep3_charge', 'lep4_charge']
training_variables += ['lep1_pdgId', 'lep2_pdgId', 'lep3_pdgId', 'lep4_pdgId']
training_variables += ['lep1_phi', 'lep2_phi', 'lep3_phi', 'lep4_phi']
training_variables += ['lep1_eta', 'lep2_eta', 'lep3_eta', 'lep4_eta']

Use again 40% of the data for training and 20% for validation. If you want to improve your training on a later point you can also use 80% for training and 20% for validation.

In [ ]:
values, weights, classification = common.get_dnn_input(data_frames, training_variables, sample_list_signal, sample_list_background, 0.6)

If you want you can play around with the batch size

In [ ]:
# Get reweighted weights
train_weights_reweighted = reweight_weights(train_weights, train_classification)
val_weights_reweighted = reweight_weights(val_weights, val_classification)

# Convert the data to tensorflow datasets


In [ ]:
# Split into train and validation data


### Model Setup

You can change the learning rate of the optimizer to improve your training

In [ ]:
# Loss function


In [ ]:
# Optimizer


In [ ]:
# Early stopping


Create your model

In [ ]:
# Normalization layer

# Create a simple NN
model_layers = [
]
model = tf.keras.models.Sequential(model_layers)
# Compile model
model.compile(optimizer=adam_optimizer, loss=loss_fn, weighted_metrics=['binary_accuracy'])

### Training

In [ ]:
# Train model
history = 

In [ ]:
# Plot the training history


In [ ]:
# Evaluate model on training and validation data


### Cross-Validation

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=3, shuffle=True, random_state=random_state)

In [ ]:
# Store the models and their training history
kfold_history = []
kfold_model = []
# Store the evaluation on training and validation data
kfold_train_eval_loss = []
kfold_train_eval_acc = []
kfold_val_eval_loss = []
kfold_val_eval_acc = []
split_idx = 1
for train_indices, val_indices in kfold.split(values):
    print(f'Use fold {split_idx}')
    

In [ ]:
# Loop over all models
for idx, model in enumerate(kfold_model):
    # Save the model
    model.save(f'models/own_model_crossval_{idx}')

In [ ]:
# Plot the training history


In [ ]:
# Evaluate the prediction on training and validation data

## Apply your Model for Higgs search

So lets use the model to measure the Higgs boson. The model prediction can be used to suppress background events.
With a selection criteria we can increase the signal purity and improve the Higgs significance.
The statistic significance of the Higgs boson can be estimated by:

$Z_{stat} = \frac{N_{Higgs}}{\sqrt{N_{bkg}}}$

In [ ]:
# Extract the values, weights, and classification for the full dataset
values, weights, classification = common.get_dnn_input(data_frames, training_variables, sample_list_signal, sample_list_background)

In [ ]:
# Split the data in signal and background
signal_values = values[classification > 0.5]
signal_weights = weights[classification > 0.5]
bkg_values = values[classification < 0.5]
bkg_weights = weights[classification < 0.5]

Lets derive the significance with the baseline model

In [ ]:
# Model prediction
signal_prediction = baseline_models[0].predict(signal_values)
bkg_prediction = baseline_models[0].predict(bkg_values)
# Transform predicton to array
signal_prediction = np.array([element[0] for element in signal_prediction])
bkg_prediction = np.array([element[0] for element in bkg_prediction])

In [ ]:
cut_value = 0.
# Number of signal and background events passing the prediction selection
n_signal = signal_weights[signal_prediction > cut_value].sum()
n_bkg = bkg_weights[bkg_prediction > cut_value].sum()

# Significance
significance = n_signal / np.sqrt(n_bkg)

print(f'The prediction selection is passed by {round(n_signal, 2)} signal and {round(n_bkg, 2)} background events.')
print(f'This results in a significance of {round(significance, 3)}')

<font color='blue'>
Task:

Calculate and plot the significances for different cuts on the prediction value. Do the calculation in a for loop and break if the number of background events is lower than 10
</font>

In [ ]:
cut_values = []
significances = []
for cut_value in np.linspace(0, 1, 1000):


In [ ]:
# Plot the significances


<font color='blue'>
Task:

What is the maximal significance you got for the model? What would be the best possible significance?
</font>

In [ ]:
print(f'The best significance by the given model is {round(max(significances), 3)}')

<font color='blue'>
Task:

What is the best significance of your model? Plot the significances dependend on the cut on the prediction value.
</font>

## Feedback

The jupyter notebook is still very work in progress. To improve this machine learning course your feedback is needed.

<font color='blue'>
Task:

What were good aspects of this notebook? What should have been more detailed and what was too much? Do you have any other suggestions for improvement?<br>
Please be honest but stay polite :)
</font>